In [1]:
import numpy as np
import pandas as pd
import math
from pysat.formula import CNF
from pysat.solvers import Solver
import random

In [2]:
data=pd.read_csv("sudoku.csv")
data

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...
...,...,...
9995,3800000006050719800090405020400650000300200780...,3845927166253719847196485322478653915361294781...
9996,9200710030760800400010090050008540903500070000...,9245716835763829418316492751628543973591274684...
9997,0080060010530800722407109300005070600050200090...,7982364511534896722467159388295471634651237893...
9998,0270060004090001070004000530040500068900000403...,5271364984395821671864792532749583168956137423...


In [3]:
num=np.random.randint(0,10000)
a=str(data['quizzes'][num])

In [4]:
sudoku=np.zeros([9,9],dtype=int)

In [5]:
for i in range(9):
    for j in range(9):
        sudoku[i][j]=int(a[i*9+j])
        
sudoku

array([[9, 0, 6, 5, 0, 3, 0, 0, 0],
       [1, 4, 0, 0, 6, 0, 0, 5, 0],
       [0, 0, 3, 0, 4, 0, 8, 7, 0],
       [0, 5, 4, 9, 7, 0, 0, 0, 2],
       [0, 3, 0, 6, 0, 1, 4, 0, 8],
       [8, 1, 9, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 5, 0, 3, 0],
       [2, 8, 7, 0, 0, 0, 0, 0, 9],
       [0, 0, 0, 4, 2, 0, 6, 0, 1]])

In [6]:
def mod(n,m):
    if n%m==0:
        return m
    else:
        return n%m

In [7]:
s=Solver()

In [8]:
r0=[]
for i in range(9):
    for j in range(9):
        if sudoku[i][j]!=0:
            for k in range(1,10):
                if i*9*9+j*9+k!=i*9*9+j*9+sudoku[i][j]:
                    r0.append(-1*(i*9*9+j*9+k))
                else:
                    r0.append(i*9*9+j*9+int(sudoku[i][j]))



In [9]:
li=[]
cnf1=CNF()
for i in range(1,9*9*9+1):
    li.append(i)
    if i%9==0:
        cnf1.append(li)
        s.add_clause(li)
        li=[]

In [10]:
cnf2=CNF()
li1=[]
for i in range(9):
    for j in range(9):
        for k in range(1,10):
            for l in range(k+1,10):
                li1=[(-1*(i*9*9+j*9+k))]
                li1.append(-1*(i*9*9+j*9+l))
                cnf2.append(li1)
                s.add_clause(li1)
                              

In [11]:
r1=[]
for i in range(1,(9*9)*(9*9)+1):
    a1=math.ceil(i/(9*9))
    b1=mod(i,9*9)
    row_a=math.ceil(a1/9)
    row_b=math.ceil(b1/9)
    col_a=mod(a1,9)
    col_b=mod(b1,9)
    block_row_a=math.ceil(row_a/3)
    block_col_a=math.ceil(col_a/3)
    block_row_b=math.ceil(row_b/3)
    block_col_b=math.ceil(col_b/3)
    if row_a==row_b:
        r1.append(9*9*9+i)
    elif col_a==col_b:
        r1.append(9*9*9+i)
    elif (block_row_a==block_row_b) and (block_col_a==block_col_b):
        r1.append(9*9*9+i)

In [12]:
cnf3=CNF()
li2=[]
for i in r1:
    i=i-729
    a1=math.ceil(i/(9*9))
    b1=mod(i,9*9)
    row_a=math.ceil(a1/9)
    row_b=math.ceil(b1/9)
    col_a=mod(a1,9)
    col_b=mod(b1,9)
    block_row_a=math.ceil(row_a/3)
    block_col_a=math.ceil(col_a/3)
    block_row_b=math.ceil(row_b/3)
    block_col_b=math.ceil(col_b/3)
    for j in range(1,10):
        if a1!=b1:
            li2.append(-1*((row_a-1)*9*9+(col_a-1)*9+j))
            li2.append(-1*((row_b-1)*9*9+(col_b-1)*9+j))
            cnf3.append(li2)
            s.add_clause(li2)
            li2=[]

In [13]:
final_assumption_list=[]
for i in r0:
    final_assumption_list.append(int(i))

In [14]:
s.solve(assumptions=final_assumption_list)

True

In [15]:
arr1=s.get_model()

In [16]:
arr2=[]
for i in range(9*9*9):
    arr2.append(arr1[i])

In [17]:
arr=np.zeros([9,9],dtype=int)
z=0
row=0
col=0
for i in arr2:
    if i>0:
        row=math.ceil(i/(9*9))-1
        col=math.ceil(mod(i,9*9)/9)-1
        arr[row][col]=mod(mod(i,9*9),9)

In [18]:
sudoku1_sol=arr
sudoku1_sol

array([[9, 7, 6, 5, 8, 3, 1, 2, 4],
       [1, 4, 8, 7, 6, 2, 9, 5, 3],
       [5, 2, 3, 1, 4, 9, 8, 7, 6],
       [6, 5, 4, 9, 7, 8, 3, 1, 2],
       [7, 3, 2, 6, 5, 1, 4, 9, 8],
       [8, 1, 9, 2, 3, 4, 7, 6, 5],
       [4, 6, 1, 8, 9, 5, 2, 3, 7],
       [2, 8, 7, 3, 1, 6, 5, 4, 9],
       [3, 9, 5, 4, 2, 7, 6, 8, 1]])